In [1]:
%pip install magic_duckdb --upgrade --quiet


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
import duckdb
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
%load_ext magic_duckdb

In [3]:
con = duckdb.connect()
con.execute("SET temp_directory='./temp_dir'")

In [4]:
%%dql -co con
summarize from './_data/G1_1e7_1e2_0_0.csv'


,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,id1,VARCHAR,id001,id100,100,None,None,None,None,None,10000000,0.0%
1,id2,VARCHAR,id001,id100,100,None,None,None,None,None,10000000,0.0%
2,id3,VARCHAR,id0000000001,id0000100000,101478,None,None,None,None,None,10000000,0.0%
3,id4,BIGINT,1,100,98,50.5099431,28.863102117454968,26,50,76,10000000,0.0%
4,id5,BIGINT,1,100,98,50.4955781,28.86326234775936,25,50,75,10000000,0.0%
5,id6,BIGINT,1,100000,99467,50011.9408402,28864.193813151654,25007,50022,75033,10000000,0.0%
6,v1,BIGINT,1,5,5,2.9998789,1.414659494528958,2,3,4,10000000,0.0%
7,v2,BIGINT,1,15,15,7.998936,4.321616912168538,4,8,12,10000000,0.0%
8,v3,DOUBLE,2.3e-05,99.999987,9679736,49.99766514080602,28.867199981503486,25.0052125814808,49.98792372366734,74.98952827822328,10000000,0.0%


In [5]:
%%dql -co con
summarize from './_data/J1_1e7_1e1_0_0.csv'

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,id1,BIGINT,1,11,10,6.4,3.2041639575194445,4,6,9,10,0.0%
1,id4,VARCHAR,id1,id9,10,None,None,None,None,None,10,0.0%
2,v2,DOUBLE,1.016213,79.701455,10,42.6621871,33.245973839596736,10.000374,39.5862765,78.482651,10,0.0%


In [6]:
%%dql -co con
summarize from './_data/J1_1e7_1e7_0_0.csv'

,column_name,column_type,min,max,approx_unique,avg,std,q25,q50,q75,count,null_percentage
0,id1,BIGINT,1,11,10,6.3999199,3.0395810068970293,4,6,9,10000000,0.0%
1,id2,BIGINT,1,11000,10093,5499.8344897,3166.505263574516,2759,5506,8235,10000000,0.0%
2,id3,BIGINT,1,10999999,10029404,5499722.3918575,3175491.3899448668,2749227,5499150,8252390,10000000,0.0%
3,id4,VARCHAR,id1,id9,10,None,None,None,None,None,10000000,0.0%
4,id5,VARCHAR,id1,id9999,9911,None,None,None,None,None,10000000,0.0%
5,id6,VARCHAR,id1,id9999999,9894656,None,None,None,None,None,10000000,0.0%
6,v2,DOUBLE,8e-06,99.999999,9556020,49.98765875129632,28.866445906585128,24.98216900988965,49.986012843875415,74.99127886512605,10000000,0.0%


In [7]:
%%dql -co con
from './_data/J1_1e7_1e7_0_0.csv'
select 
    count(*),
    count(distinct id3),
    count(distinct id6)

,count_star(),count(DISTINCT id3),count(DISTINCT id6)
0,10000000,10000000,10000000


In [8]:
%%dql -co con
create or replace table big as
from './_data/J1_1e7_1e7_0_0.csv'

,Count
0,10000000


## Types of Window Functions
- over ()
- first() over (order by )
- row_number() over (order by )
- partition, no order
    - tons of tiny partitions
- partition and order
- order by and rows between 1 preceding and 1 preceding (lag)
- order by and rows between unbounded preceding and current row
- order by and range between (value - 3) preceding and current row
- All of the frame specs, but with a partition

In [10]:
%%dql -co con
from big 
select 
    id1,
    id2,
    id3,
    v2,
    -- Benchmark an aggregate on the whole dataset
    sum(v2) over (),

    -- Benchmark the sorting alone
    first(v2) over (order by id3),

    -- Typical row_number
    row_number() over (order by id3),

    -- Sum over partition by
    sum(v2) over (partition by id1),
    sum(v2) over (partition by id2),
    sum(v2) over (partition by id3),

    -- Benchmark sorting within partitions
    first(v2) over (partition by id2 order by id3),

    quantile_cont(v2, [0, 0.25, 0.50, 0.75, 1]) over (),

    -- Lag
    first(v2) over (order by id3 rows between 1 preceding and 1 preceding),

    -- Moving average over last 100
    avg(v2) over (order by id3 rows between 100 preceding and current row),
    -- Moving average over last id1 rows
    avg(v2) over (order by id3 rows between id1 preceding and current row),

    -- Rolling Sum
    sum(v2) over (order by id3 rows between unbounded preceding and current row),

    -- Range between a constant
    sum(v2) over (order by v2 range between 3 preceding and current row),
    -- Range between id1 values away
    sum(v2) over (order by v2 range between id1 preceding and current row),

    -- Plus partitioning:
    quantile_cont(v2, [0, 0.25, 0.50, 0.75, 1]) over (partition by id2),

    -- Lag
    first(v2) over (partition by id2 order by id3 rows between 1 preceding and 1 preceding),

    -- Moving average over last 100
    avg(v2) over (partition by id2 order by id3 rows between 100 preceding and current row),
    -- Moving average over last id1 rows
    avg(v2) over (partition by id2 order by id3 rows between id1 preceding and current row),

    -- Rolling Sum
    sum(v2) over (partition by id2 order by id3 rows between unbounded preceding and current row),

    -- Range between a constant
    sum(v2) over (partition by id2 order by v2 range between 3 preceding and current row),
    -- Range between id1 values away
    sum(v2) over (partition by id2 order by v2 range between id1 preceding and current row),

    quantile_cont(v2, [0, 0.25, 0.50, 0.75, 1]) over (partition by id2 order by id3 rows between 100 preceding and current row),

order by id2, v2




,id1,id2,id3,v2,sum(v2) OVER (),first(v2) OVER (ORDER BY id3),row_number() OVER (ORDER BY id3),sum(v2) OVER (PARTITION BY id1),sum(v2) OVER (PARTITION BY id2),sum(v2) OVER (PARTITION BY id3),first(v2) OVER (PARTITION BY id2 ORDER BY id3),"quantile_cont(v2, main.list_value(0, 0.25, 0.50, 0.75, 1)) OVER ()",first(v2) OVER (ORDER BY id3 ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING),avg(v2) OVER (ORDER BY id3 ROWS BETWEEN 100 PRECEDING AND CURRENT ROW),avg(v2) OVER (ORDER BY id3 ROWS BETWEEN id1 PRECEDING AND CURRENT ROW),sum(v2) OVER (ORDER BY id3 ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW),sum(v2) OVER (ORDER BY v2 RANGE BETWEEN 3 PRECEDING AND CURRENT ROW),sum(v2) OVER (ORDER BY v2 RANGE BETWEEN id1 PRECEDING AND CURRENT ROW),"quantile_cont(v2, main.list_value(0, 0.25, 0.50, 0.75, 1)) OVER (PARTITION BY id2)",first(v2) OVER (PARTITION BY id2 ORDER BY id3 ROWS BETWEEN 1 PRECEDING AND 1 PRECEDING),avg(v2) OVER (PARTITION BY id2 ORDER BY id3 ROWS BETWEEN 100 PRECEDING AND CURRENT ROW),avg(v2) OVER (PARTITION BY id2 ORDER BY id3 ROWS BETWEEN id1 PRECEDING AND CURRENT ROW),sum(v2) OVER (PARTITION BY id2 ORDER BY id3 ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW),sum(v2) OVER (PARTITION BY id2 ORDER BY v2 RANGE BETWEEN 3 PRECEDING AND CURRENT ROW),sum(v2) OVER (PARTITION BY id2 ORDER BY v2 RANGE BETWEEN id1 PRECEDING AND CURRENT ROW),"quantile_cont(v2, main.list_value(0, 0.25, 0.50, 0.75, 1)) OVER (PARTITION BY id2 ORDER BY id3 ROWS BETWEEN 100 PRECEDING AND CURRENT ROW)"
0,10,1,3852128,0.190069,4.998766e+08,45.388357,3502390,4.999822e+07,48957.096388,0.190069,47.639135,"[8e-06, 24.98767875, 49.992937, 74.98793575, 9...",99.298290,47.289939,34.837900,1.751520e+08,1.846906e+03,1.846906e+03,"[0.190069, 24.818931, 51.09455, 75.735511, 99....",47.397744,48.681893,38.073053,15533.260411,0.190069,0.190069,"[0.190069, 23.481975, 53.546683, 80.341311, 97..."
1,8,1,2299458,0.313571,4.998766e+08,45.388357,2090601,5.004526e+07,48957.096388,0.313571,47.639135,"[8e-06, 24.98767875, 49.992937, 74.98793575, 9...",76.886541,53.495397,31.902307,1.046104e+08,4.974486e+03,4.974486e+03,"[0.190069, 24.818931, 51.09455, 75.735511, 99....",4.250971,46.078051,29.147291,9449.343807,0.503640,0.503640,"[0.313571, 27.413748, 60.044402, 23.887305, 99..."
2,4,1,9688352,0.320398,4.998766e+08,45.388357,8807775,4.999699e+07,48957.096388,0.320398,47.639135,"[8e-06, 24.98767875, 49.992937, 74.98793575, 9...",70.221374,47.984421,38.951024,4.402999e+08,5.190939e+03,5.190939e+03,"[0.190069, 24.818931, 51.09455, 75.735511, 99....",2.503148,48.408545,14.438150,42627.807918,0.824038,0.824038,"[0.320398, 26.694021, 80.166097, 9.100832, 98...."
3,8,1,2109944,0.408771,4.998766e+08,45.388357,1918388,5.004526e+07,48957.096388,0.408771,47.639135,"[8e-06, 24.98767875, 49.992937, 74.98793575, 9...",91.319090,55.565313,61.313023,9.599227e+07,8.381171e+03,8.381171e+03,"[0.190069, 24.818931, 51.09455, 75.735511, 99....",58.058500,46.721475,39.478608,8681.306533,1.232809,1.232809,"[0.408771, 27.413748, 60.044402, 23.887305, 99..."
4,10,1,9960494,0.512296,4.998766e+08,45.388357,9055235,4.999822e+07,48957.096388,0.512296,47.639135,"[8e-06, 24.98767875, 49.992937, 74.98793575, 9...",5.453245,49.905788,41.643999,4.526577e+08,1.319554e+04,1.319554e+04,"[0.190069, 24.818931, 51.09455, 75.735511, 99....",86.609923,47.487036,45.226255,43491.632431,1.745105,1.745105,"[0.320398, 24.818931, 0.512296, 72.820091, 99...."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9999995,9,11000,5465455,99.689424,4.998766e+08,45.388357,4969188,4.995443e+07,50024.545796,99.689424,87.197532,"[8e-06, 24.98767875, 49.992937, 74.98793575, 9...",36.218034,56.394777,57.879934,2.484624e+08,2.935440e+07,8.548234e+07,"[0.007788, 24.289348500000003, 51.042906, 76.2...",24.877720,50.688090,50.207974,24609.737869,2269.402431,7984.952949,"[0.007788, 45.063237, 25.733251, 3.801425, 99...."
9999996,10,11000,6004715,99.754183,4.998766e+08,45.388357,5459206,4.9998